### Memory Management

#### Memory is a service that provides a long-term knowledge storage for the agents.

Session = Short-term memory (single conversation)(like application state - temporary)

Memory = Long-term knowledge (across multiple conversations)(like database - persistent)

In [1]:
import os

try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Setup complete.")
except Exception as e:
  print("Error: ",e)


Setup complete.


In [2]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("ADK components imported successfully.")

ADK components imported successfully.


In [9]:
async def run_session(
    runner_instance:Runner,
    user_queries: list[str] |str,
    session_id: str="default"
):
  """Helper function to run queries in a session and display responses."""
  print(f"\n ### Session:{session_id}")

  try:
    session = await session_service.create_session(
      app_name=APP_NAME,user_id=USER_ID,session_id = session_id
    )
  except:
    session = await session_service.get_service(
      app_name = APP_NAME, user_id=USER_ID,session_id= session_id
    )

  #Convert single query to list
  if isinstance(user_queries,str):
    user_queries = [user_queries]

  #process each query
  for query in user_queries:
    print(f"\n User>{query}")
    query_content = types.Content(role="user",parts=[types.Part(text=query)])

    #stream agent response
    async for event in runner_instance.run_async(
      user_id=USER_ID,session_id=session_id,new_message=query_content
    ):
      if event.is_final_response() and event.content and event.content.parts:
        text = event.content.parts[0].text
        if text and text != "None":
          print(f"Model: > {text}")

### Memory Workflow

1. initialize -> create a `MemoryService` & provide it to the agent via `Runner`
2. ingest -> transfer session data to memory using `add_session_to_memory()`
3. retrieve -> search stored memories using `search_memory()`

In [10]:
memory_service = (
  InMemoryMemoryService() #adk's built in memory service for dev and testing
)                         # creates the space into the RAM

In [11]:
retry_config = types.HttpRetryOptions(
  exp_base=7,
  initial_delay=1,
  attempts=5,
  http_status_codes=[429,500,503,504]
)

In [12]:
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

user_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name="MemoryDemoAgent",
  instruction="Answer user questions in simple words."
)


In [13]:
session_service = InMemorySessionService() #handles conversations

runner = Runner(
  agent=user_agent,
  app_name= APP_NAME,
  session_service=session_service,
  memory_service= memory_service,
)

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Adding `memory_service` to the `runner` makes memory available to the agent, but doesnt automatically uses it. It has to:
1. ingest data using `add_session_to_memory()`
2. enable retrieval by giving the agent memory tools (`load_memory` or `preload_memory`)

In [14]:
await run_session(
  runner,
  "My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? ",
  "conversation-01" #sessionID
)


 ### Session:conversation-01

 User>My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? 
Model: > That's a great motto! Here are a few alternative ways to say "Nothing easy is worth doing":

*   **The best things take effort.**
*   **Hard work leads to great rewards.**
*   **If it's too simple, it's probably not that valuable.**
*   **Challenges make accomplishments meaningful.**
*   **Don't shy away from a tough task.**


In [16]:
#Lets verify the conversation was captured in the session

session = await session_service.get_session(
  user_id=USER_ID, app_name=APP_NAME,session_id="conversation-01")

In [20]:
session.events[0]

Event(model_version=None, content=Content(
  parts=[
    Part(
      text="My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? "
    ),
  ],
  role='user'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-6bb3ae7c-6a15-414a-b9f4-2fc2b247604e', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=None, end_of_agent=None, agent_state=None, rewind_before_invocation_id=None), long_running_tool_ids=None, branch=None, id='029ddc08-ac73-401b-b911-d2ba63949896', timestamp=1

In [23]:
print("Session Contains:")

for event in session.events:
  text = (
    event.content.parts[0].text[0:200]
    if event.content and event.content.parts
    else "empty"
  )
  print(f"{event.content.role} : {text}")

Session Contains:
user : My favourite moto is 'Nothing easy is worth doing', can you think of an alternative way of expressing it? 
model : That's a great motto! Here are a few alternative ways to say "Nothing easy is worth doing":

*   **The best things take effort.**
*   **Hard work leads to great rewards.**
*   **If it's too simple, it


In [24]:
# transfering the conversation into memory with add_session_to_memory()

await memory_service.add_session_to_memory(session)

In [25]:
# adding load memory tool to the agent

user_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name = "MemoryDemoAgent",
  description="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
  tools=[load_memory]
)

In [26]:
# updating the runner also
runner = Runner(
  agent=user_agent,
  app_name=APP_NAME,
  session_service=session_service,
  memory_service=memory_service,
)

App name mismatch detected. The runner is configured with app name "MemoryDemoApp", but the root agent was loaded from "/Users/liakooras/Desktop/UDEMIES/genAiKaggle/.conda/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


In [28]:
await run_session(runner,"Whats my favourite motto?","motto test")


 ### Session:motto test

 User>Whats my favourite motto?
